In [4]:
import requests
import pendulum
import datetime


#https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot
#&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor,MODIS_Terra_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME=2023-01-12&WRAP=DAY,DAY&
#BBOX=33.451,-103.4285,37.169,-94.0015&FORMAT=image/jpeg&WIDTH=1073&HEIGHT=423&AUTOSCALE=TRUE&ts=1699388283201
#bottom left latitide, bottom left longitude, top right latitude, top right logitude

#https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor,MODIS_Terra_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME=2023-01-12&WRAP=DAY,DAY&BBOX=33.451,-103.4285,37.169,-94.0015&FORMAT=image/jpeg&WIDTH=1073&HEIGHT=423&AUTOSCALE=TRUE
#&ts=1699388957535

#BASE_URL = "https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME={time}&BBOX={bbox}&CRS=EPSG:4326&LAYERS={layers}&FORMAT={format}&WIDTH={width}&HEIGHT={height}"
#BASE_URL = "https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=VIIRS_NOAA20_CorrectedReflectance_TrueColor,VIIRS_NOAA20_Thermal_Anomalies_375m_Day&CRS=EPSG:4326&TIME=2023-06-14&WRAP=DAY,&BBOX=44.1105,-80.8925,63.4595,-55.9775&FORMAT=image/jpeg&WIDTH=2835&HEIGHT=2201&AUTOSCALE=TRUE&ts=1698602072041"
#BASE_URL = "https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor&CRS=EPSG:4326&TIME=2023-10-29&WRAP=DAY&BBOX=40.921,-96.201,57.619,-73.299&FORMAT=image/jpeg&WIDTH=521&HEIGHT=380&AUTOSCALE=TRUE&ts=1698602634467"
#BASE_URL = "https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor,MODIS_Terra_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME=2023-09-05&WRAP=DAY,DAY&BBOX=33.451,-103.4285,37.169,-94.0015&FORMAT=image/jpeg&WIDTH=215&HEIGHT=85&AUTOSCALE=TRUE&ts=1698603368438"
#BASE_URL = "https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor,MODIS_Terra_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME=2023-01-12&WRAP=DAY,DAY&BBOX=33.451,-103.4285,37.169,-94.0015&FORMAT=image/jpeg&WIDTH=1073&HEIGHT=423&AUTOSCALE=TRUE&ts=1698603510281"
# example arguments:
#  time: 2020-01-29T12:10:00Z
#  bbox: 10.177734375,-60.1875,16.41796875,-53.5166015625
#  layers: GOES-East_ABI_Band2_Red_Visible_1km,Reference_Labels
#  format: image/tiff
#  width: 1518
#  height: 1420



#Latitude
#33.25559 °

#Longitude
#-100.40601 °

def download_image_url(fn, time, bbox, layers, image_format, resolution):
    """
    example arguments:
     time: datetime(2020-01-29T12:10:00Z)
     bbox: [10.177734375, -60.1875, 16.41796875, -53.5166015625]
     layers: ["GOES-East_ABI_Band2_Red_Visible_1km", "Reference_Labels"]
     image_format: "tiff"
     resolution: 0.01 (deg/pixel)
    """
    BASE_URL = "https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Aqua_CorrectedReflectance_TrueColor,MODIS_Aqua_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME={date}&WRAP=DAY,DAY&BBOX={bbox}&FORMAT=image/jpeg&WIDTH=5670&HEIGHT=4403&AUTOSCALE=TRUE&ts={timestamp}"
    
    if len(bbox) != 4:
        raise Exception("`bbox` should be a list of [lat_S, lon_W, lat_N, lon_E]")

    if time.tzinfo is None:
        raise Exception("The provided timestamp doesn't contain timezone info")

    # round time to nearest second
    time = time - datetime.timedelta(microseconds=time.microsecond)

    #dlon = bbox[2]-bbox[0]
    #dlat = bbox[3]-bbox[1]
    #width, height = int(dlat/resolution), int(dlon/resolution)

    url = BASE_URL.format(
        time=pendulum.instance(time).to_iso8601_string(),
        bbox=",".join([str(v) for v in bbox]),
        #layers=",".join(layers),
        #format="image/{}".format(image_format),
        #width=width,
        #height=height
    )
    #r = requests.get(url)
    #print(r)
    #if r.status_code == 200:
    #    if 'xml' in r.text[:40]:
    #        print(url)
    #        raise Exception(r.content)
    #    else:
    #        with open(fn, 'wb') as fh:
    #            fh.write(r.content)
    #else:
    #    raise Exception(r.status)

def download_img():
    r = requests.get(BASE_URL)
    if r.status_code == 200:
        if 'xml' in r.text[:40]:
            print(url)
            raise Exception(r.content)
        else:
            with open('image7.png', 'wb') as fh:
                fh.write(r.content)
    else:
        raise Exception(r.status)

#download_img()

In [ ]:
def group_fire_data_xarray(file_name):
    file_path = "./historical_fire_data/" + file_name + ".csv"
    fire_data = pd.read_csv(file_path)
    
    filtered_fire_data = fire_data.query("`confidence` >= 75 and `type` == 0")
    grouped_fire_data = filtered_fire_data.groupby(['acq_date', 'daynight']).agg({'latitude':'mean','longitude':'mean','brightness':'mean','confidence':'mean', 'frp':'mean', 'bright_t31':'mean'}).reset_index()

    for modis_data_index, modis_row in grouped_fire_data.iterrows():
        #print(modis_row['acq_date'])
        date_value = modis_row['acq_date']
        confidence = modis_row['confidence']
        latitude = modis_row['latitude']
        longitude = modis_row['longitude']
        day_night_ind = modis_row['daynight']
        #print(type(confidence))
        if confidence > 75:
            north_coord = latitude+0.25
            south_coord = latitude-0.25
            west_coord = longitude-0.25
            east_coord = longitude+0.25
            area = [south_coord, west_coord, north_coord, east_coord]
